In [3]:
import numpy as np
from tqdm import tqdm
from solver.qp import qp
import torch
import gzip
import pickle
import warnings
from generate_instances import generate_softmarginsvm

### Soft margin Support Vector Machine

In [16]:
def surrogate_gen():
    C = 10 #regularization parameter
    m = 4 #number of Datapoints
    X = np.random.uniform(-1, 1, (m, 2)) #Dataset
    bias = np.full((m, 1), 0.5)
    X[:, 1] += bias.ravel()   # Adding a bias to make data linearly separable
    y = np.array([1.0 if x[0] > x[1] else -1.0 for x in X]).reshape(m, 1) #label vector
    X_dash = (y.reshape(-1,1) * 1.) * X #X_dash * X_dash^T = Q
    Q, q, G, h, A, b = generate_softmarginsvm(y, X, X_dash, C)    
    return Q, q, G, h, A, b, X_dash

### create

In [25]:
import os

directory = f'Quadratic_Programming_Datasets/raw'
os.makedirs(directory, exist_ok=True)
warnings.filterwarnings("error")

ips = []
pkg_idx = 0 #identifier index
success_cnt = 0
fail_cnt = 0

max_iter = 2
num = 2

for i in tqdm(range(max_iter)):
    Q, q, G, h, A, b, S = surrogate_gen()
    
    try:
        res = qp(Q, q, G, h, A, b)
        print("status",res['status'])
    except Exception as e:
        fail_cnt += 1
        warnings.warn(f'Optimization failed with error: {str(e)}')
        continue
    else:
        if res['status'] == 'optimal': #np.isnan(res.fun)?
            ips.append((torch.from_numpy(np.array(Q)).to(torch.float), torch.from_numpy(np.array(q)).to(torch.float), torch.from_numpy(np.array(G)).to(torch.float), torch.from_numpy(np.array(h)).to(torch.float), torch.from_numpy(np.array(A)).to(torch.float), torch.from_numpy(np.array(b)).to(torch.float), torch.from_numpy(np.array(S)).to(torch.float)))
            print(success_cnt)
            success_cnt += 1
            
    if len(ips) >= 1000 or success_cnt == num:
        print("success")
        with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            pkg_idx += 1
        ips = []

    if success_cnt >= num:
        break

warnings.resetwarnings()

 50%|█████     | 1/2 [00:00<00:00, 29.00it/s]

     pcost       dcost       gap    pres   dres
 0:  1.5040e+01 -9.8346e+01  1e+02  1e-15  1e-15
 1:  1.4951e+00 -1.1130e+01  1e+01  7e-16  1e-15
 2: -1.5197e+00 -2.9771e+00  1e+00  1e-16  2e-16
 3: -1.7666e+00 -1.9278e+00  2e-01  4e-16  1e-16
 4: -1.8362e+00 -1.8516e+00  2e-02  3e-16  1e-16
 5: -1.8414e+00 -1.8423e+00  9e-04  2e-16  8e-17
 6: -1.8415e+00 -1.8415e+00  1e-05  5e-16  3e-17
 7: -1.8415e+00 -1.8415e+00  1e-07  1e-15  5e-17
Optimal solution found.
status optimal
0
     pcost       dcost       gap    pres   dres
 0:  1.0759e+01 -1.1228e+02  1e+02  9e-16  9e-16
 1:  2.0693e+00 -1.2087e+01  1e+01  7e-17  2e-15
 2: -1.0959e+00 -2.6779e+00  2e+00  3e-16  6e-16
 3: -1.3687e+00 -1.5023e+00  1e-01  2e-16  2e-16
 4: -1.4120e+00 -1.4152e+00  3e-03  3e-16  8e-17
 5: -1.4132e+00 -1.4133e+00  3e-05  2e-16  5e-17
 6: -1.4132e+00 -1.4132e+00  3e-07  2e-16  2e-16
Optimal solution found.
status optimal
1
success


In [20]:
# Open and load the .pkl.gz file
with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{pkg_idx-1}.pkl.gz', 'rb') as f:
    data = pickle.load(f)

# Now `data` contains the contents of the pickle file
print(data)

[(tensor([[ 0.4303,  0.0430, -0.3151,  0.0879],
        [ 0.0430,  0.0088, -0.0982, -0.0740],
        [-0.3151, -0.0982,  1.2148,  1.1568],
        [ 0.0879, -0.0740,  1.1568,  1.5333]]), tensor([[-1.],
        [-1.],
        [-1.],
        [-1.]]), tensor([[-1., -0., -0., -0.],
        [-0., -1., -0., -0.],
        [-0., -0., -1., -0.],
        [-0., -0., -0., -1.],
        [ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]), tensor([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [10.],
        [10.],
        [10.],
        [10.]]), tensor([[-1., -1., -1., -1.]]), tensor([[0.]])), (tensor([[2.1528, 0.3875, 2.0707, 0.7480],
        [0.3875, 0.2027, 0.2725, 0.4138],
        [2.0707, 0.2725, 2.0674, 0.5088],
        [0.7480, 0.4138, 0.5088, 0.8464]]), tensor([[-1.],
        [-1.],
        [-1.],
        [-1.]]), tensor([[-1., -0., -0., -0.],
        [-0., -1., -0., -0.],
        [-0., -0., -1., -0.],
        [-0., 